# Conversando com os dados

In [1]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
caminhos = [
    "arquivos/Explorando a API da OpenAI.pdf",
    ]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [3]:
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings()
)

## Criando estrutura de conversa

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    '''Responda as perguntas se baseando no contexto fornecido.
    
    contexto: {contexto}
    
    pergunta: {pergunta}''')

In [5]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 25})
setup = RunnableParallel({
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
})

In [6]:
input = setup.invoke('O que é a OpenAI?')
input

{'pergunta': 'O que é a OpenAI?',
 'contexto': [Document(id='a3810ac7-b873-4b64-aea9-b25d3ab6b50b', metadata={'source': 'Explorando a API da OpenAI.pdf', 'page': 9, 'doc_id': 19}, page_content='Explorando a API da OpenAI\nGuias\nTambém está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da\nOpenAI.\nAsimov Academy 9'),
  Document(id='fda315f2-e4c5-48c5-bf6b-983e3b3f231f', metadata={'source': 'Explorando a API da OpenAI.pdf', 'page': 6, 'doc_id': 15}, page_content='Explorando a API da OpenAI\nA OpenAI\nNosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência\nartificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela\nempresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:\n• GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multimodal, que aceita\nentradas de texto ou imagem e produz texto. Ele é capaz de resolver problemas c

In [7]:
def join_documents(input):
    input['contexto'] = '\n\n'.join([c.page_content for c in input['contexto']])
    return input

setup = RunnableParallel({
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
}) | join_documents

In [8]:
input = setup.invoke('O que é a OpenAI?')
print(input['contexto'])

Explorando a API da OpenAI
Guias
Também está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da
OpenAI.
Asimov Academy 9

Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela
empresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:
• GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multimodal, que aceita
entradas de texto ou imagem e produz texto. Ele é capaz de resolver problemas complexos com
maior precisão do que os modelos anteriores da OpenAI.
• DALL-E: Este sistema de IA pode criar imagens e arte realistas a partir de descrições textuais. O
DALL·E 3, em particular, suporta a capacidade de gerar novas imagens em tamanhos específicos,
com base em sugestões fornecidas.
• TTS: É um modelo de IA que transforma texto em fala com 

In [9]:
from langchain_openai import ChatOpenAI

chain = setup | prompt | ChatOpenAI()
chain.invoke('O que é a OpenAI?')

AIMessage(content='A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo na atualidade, que cria diversos modelos de IA, como o GPT-4, DALL-E, TTS e Whisper.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 1090, 'total_tokens': 1137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9b0125b6-e600-43bf-990c-6277d69128cc-0', usage_metadata={'input_tokens': 1090, 'output_tokens': 47, 'total_tokens': 1137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Juntando tudo

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    '''Responda as perguntas se baseando no contexto fornecido.
    
    contexto: {contexto}
    
    pergunta: {pergunta}''')

retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 25})

def join_documents(input):
    input['contexto'] = '\n\n'.join([c.page_content for c in input['contexto']])
    return input

setup = RunnableParallel({
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
}) | join_documents

chain = setup | prompt | ChatOpenAI()
chain.invoke('O que é a OpenAI?')

AIMessage(content='A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo na atualidade. Ela cria diversos modelos de IA, como o GPT-4, DALL-E, TTS, e Whisper, que possuem diferentes funções, como produção de texto, criação de imagens a partir de descrições textuais, transformação de texto em fala, reconhecimento de fala, entre outros.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 1090, 'total_tokens': 1183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1024}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-12b57f8c-a24b-4d55-9171-5f517a6f79fa-0', usage_metadata={'input_tokens': 1090, 'output_tokens': 93, 'total_tokens': 1183, 'input_token_details': {'audio': 0, 'cach